In [1]:
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D


In [2]:
mnist = tf.keras.datasets.mnist

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
X_train = X_train / 255.0
X_test = X_test / 255.0

X_train = X_train[..., tf.newaxis]
X_test = X_test[..., tf.newaxis]

11490434/11490434 [==============================] - 0s 0us/step


In [3]:
X_train[0].shape

(28, 28, 1)

In [4]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).shuffle(1000).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test)).batch(32)

In [11]:
class MyModel(tf.keras.Model):
  def __init__(self):
    super().__init__()
    self.conv1 = Conv2D(32, 3, activation = "relu")
    self.flatten = Flatten()
    self.d1 = Dense(128, activation = "relu")
    self.d2 = Dense(10)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    x = self.d2(x)
    return x

model = MyModel()



In [12]:
loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)
optimizer = tf.keras.optimizers.Adam()


In [13]:
train_loss = tf.keras.metrics.Mean()
test_loss = tf.keras.metrics.Mean()

train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

In [14]:
@tf.function
def train(images, labels):
  with tf.GradientTape() as tape:
      pred = model(images)
      loss = loss_function(labels, pred)

  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  train_loss(loss)
  train_accuracy(labels, pred)


In [15]:
@tf.function
def test(images, labels):
  pred = model(images)
  loss = loss_function(labels, pred)
  test_loss(loss)
  test_accuracy(labels, pred)


In [17]:
epochs = 5

for epoch in tqdm(range(epochs)):
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  #train
  for images, labels in train_dataset:
    train(images, labels)


  #test
  for images,labels in test_dataset:
    test(images, labels)

  print(f"Epoch : {epoch+1} , Train Loss : {train_loss.result()} , Train Acuuracy : {train_accuracy.result()} , Test Loss : {test_loss.result()} , Test Accuracy : {test_accuracy.result()}")

 20%|██        | 1/5 [00:04<00:19,  4.77s/it]

Epoch : 1 , Train Loss : 0.006404635030776262 , Train Acuuracy : 0.9979833364486694 , Test Loss : 0.07669026404619217 , Test Accuracy : 0.9815000295639038


 40%|████      | 2/5 [00:10<00:15,  5.22s/it]

Epoch : 2 , Train Loss : 0.005210361443459988 , Train Acuuracy : 0.9984666705131531 , Test Loss : 0.0756436437368393 , Test Accuracy : 0.9829000234603882


 60%|██████    | 3/5 [00:15<00:10,  5.05s/it]

Epoch : 3 , Train Loss : 0.004347135778516531 , Train Acuuracy : 0.9986000061035156 , Test Loss : 0.07171289622783661 , Test Accuracy : 0.9837999939918518


 80%|████████  | 4/5 [00:19<00:04,  4.72s/it]

Epoch : 4 , Train Loss : 0.0037897424772381783 , Train Acuuracy : 0.9987000226974487 , Test Loss : 0.06990133970975876 , Test Accuracy : 0.9843999743461609


100%|██████████| 5/5 [00:23<00:00,  4.76s/it]

Epoch : 5 , Train Loss : 0.0029801269993185997 , Train Acuuracy : 0.9990500211715698 , Test Loss : 0.08969840407371521 , Test Accuracy : 0.982699990272522


In [21]:
import cv2
import numpy as np

image = cv2.imread("/content/3.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
image = cv2.resize(image, (28, 28))
image = image[...,tf.newaxis]
image = image[tf.newaxis, ...]
image = image.astype("float32")
image.shape
#batch_size, width, height, channel

(1, 28, 28, 1)

In [22]:
pred = model(image)
result = np.argmax(pred)
result


3

In [23]:
model.save("mnist_model")

In [26]:
model = tf.keras.models.load_model("mnist_model")